# Extract Data in Chunks 

This notebook extracts open data in 30 day spans. This is to be used for question answering training.

In [1]:
import pandas as pd
import datetime

In [2]:
earliest = pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=MIN(call_date)")
latest = pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=MAX(call_date)")
earliest = pd.to_datetime(earliest.iloc[0,0])
latest = pd.to_datetime(latest.iloc[0,0])
print('The records are between',earliest,' and ',latest,' .')

The records are between 2000-04-12 00:00:00  and  2022-01-26 00:00:00  .


In [13]:
range_date = earliest+pd.DateOffset(days=30)
fn_start='data/sf_cfs_'
fn_end='.csv'

while range_date < latest:
    earliest_iso=earliest.isoformat()
    range_iso=range_date.isoformat()
    num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +earliest_iso+"%27%20AND%20%27"+range_iso+"%27")
    maxNumber=str(num.iloc[0,0])
    query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +earliest_iso+"%27%20AND%20%27"
           +range_iso+"%27&$limit="+maxNumber)
    cfs_data=pd.read_json(query_str)
    fn=fn_start+str(earliest)[0:10]+'_'+str(range_date)[0:10]+fn_end
    cfs_data.to_csv(fn,index=False)
    earliest= earliest+pd.DateOffset(days=30)
    range_date = range_date+pd.DateOffset(days=30)
    
    